In [1]:
%load_ext autoreload
import sys
sys.path.append('../..')
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os, shutil, cv2
import re

In [2]:
dt_all = pd.read_pickle('meta_data_video.pkl')
dt_all.columns

Index(['embeddings', 'frames', 'total_frames', 'faces_samples', 'video',
       'super_cluster', 'video_color'],
      dtype='object')

In [3]:
pct = dt_all.frames.apply(len)/dt_all.total_frames
dt_all['pct'] = pct
dt_pct = pd.DataFrame(dt_all.groupby(['video', 'super_cluster']).sum()['pct']).reset_index()
dt_pct.video = dt_pct.video.apply(lambda x: x.split('/')[-1])
dt_pct['video_id'] = dt_pct.video.apply(lambda x: int(re.findall(r'\d+', x)[0]))

In [4]:
dt_desc = pd.read_csv('descomplica.csv')
dt_desc = dt_desc.loc[dt_desc.name_saved.apply(lambda x: 'v' in x)]
dt_desc['video_id'] = dt_desc.name_saved.apply(lambda x: int(re.findall(r'\d+', x)[0]))
dt_desc = dt_desc.set_index('video_id')

In [5]:
simi = {}
for id1 in dt_desc.index:
    cur = []
    for id2 in dt_desc.index:
        if bool(set(dt_desc.loc[id1].prof_names.strip().split(', ')) 
                & set(dt_desc.loc[id2].prof_names.strip().split(', '))):
            cur.append(id2)
    simi[id1] = cur

In [6]:
dt_desc['rec'] = pd.Series(simi)
dt_desc.to_csv('descomplica_tunado.csv')
dt_desc.head()

,prof_number,prof_names,url,name_saved,rec
video_id,,,,,
73,1,Rafael Cunha,https://www.youtube.com/watch?v=Aa660097lq0,v (73),"[73, 7, 60, 75, 53, 1, 98, 45, 79, 47]"
7,3,"Rafael Cunha, Mafê, Debora Aladim",https://www.youtube.com/watch?v=A19cwnht4Rs,v (7),"[73, 7, 60, 75, 53, 1, 98, 45, 79, 47, 83, 74,..."
60,1,Rafael Cunha,https://www.youtube.com/watch?v=V0OthZxiMOk,v (60),"[73, 7, 60, 75, 53, 1, 98, 45, 79, 47]"
75,5,"Rafael Cunha, Eduardo Valadares, Claúdio Hanse...",https://www.youtube.com/watch?v=ZC41QK_HVgI,v (75),"[73, 7, 60, 75, 53, 1, 98, 45, 79, 47, 74, 69,..."
53,1,Rafael Cunha,https://www.youtube.com/watch?v=PHBQx6TUNrU,v (53),"[73, 7, 60, 75, 53, 1, 98, 45, 79, 47]"


In [7]:
dt_pct.head()

,video,super_cluster,pct,video_id
0,v (1).mp4,7,1.016393,1
1,v (10).mp4,1,0.431624,10
2,v (10).mp4,18,0.610125,10
3,v (11).mp4,9,0.754564,11
4,v (12).mp4,4,0.857143,12


## Recommendation

In [8]:
from sklearn.metrics import average_precision_score

In [9]:
def average_precision_ranking(rank, video_id):
    y_true = []
    y_score = []
    
    for i, r in enumerate(rank):
        y_score.append(100-i)
        y_true.append(int(r in dt_desc.loc[video_id].rec))
    
    return average_precision_score(y_true, y_score)

In [29]:
results = {}
for i in range(21):
    dt_pct_f = dt_pct.loc[dt_pct.pct >= (i/100)].copy()
    
    average_precision = []

    for video_id in dt_desc.index:
        #pct de cada pessoa
        actor_score = dt_pct_f.loc[dt_pct_f.video_id == video_id][['super_cluster','pct']].set_index('super_cluster')
        #as pessoas
        actors = dt_pct_f.loc[dt_pct_f.video_id == video_id].super_cluster.values
        #pct das pessoas em outro video
        dt_score = dt_pct_f.loc[(dt_pct_f.super_cluster.isin(actors))&(dt_pct_f.video_id != video_id)].copy()
        #score (multiplicacao)
        dt_score['score'] = dt_score.super_cluster.map(actor_score.pct)*dt_score.pct
        #rank com a soma de quem aparece
        rank = dt_score.groupby('video_id')['score'].sum().nlargest(100).index.values
        #average precision
        ap = average_precision_ranking(rank, video_id)

        #print(f'id {video_id} = ', ap)


        average_precision.append(ap)
        
    results[i] = [np.mean(average_precision), np.std(average_precision), np.min(average_precision)]
    print(f'{i}%: MAP: {results[i][0]} stdAP: {results[i][1]} MinAP: {results[i][2]}')

0%: MAP: 0.9605228442128014 stdAP: 0.09557796616018162 MinAP: 0.45464362656623014
1%: MAP: 0.9605228442128014 stdAP: 0.09557796616018162 MinAP: 0.45464362656623014
2%: MAP: 0.9605228442128014 stdAP: 0.09557796616018162 MinAP: 0.45464362656623014
3%: MAP: 0.9605358929105415 stdAP: 0.09554171595271742 MinAP: 0.45464362656623014
4%: MAP: 0.9605358929105415 stdAP: 0.09554171595271742 MinAP: 0.45464362656623014
5%: MAP: 0.9605358929105415 stdAP: 0.09554171595271742 MinAP: 0.45464362656623014
6%: MAP: 0.9605358929105415 stdAP: 0.09554171595271742 MinAP: 0.45464362656623014
7%: MAP: 0.9606239915272758 stdAP: 0.09525036152750999 MinAP: 0.45695976173173014
8%: MAP: 0.9606342778998894 stdAP: 0.0952224832838032 MinAP: 0.45695976173173014
9%: MAP: 0.9606342778998894 stdAP: 0.0952224832838032 MinAP: 0.45695976173173014
10%: MAP: 0.9606342778998894 stdAP: 0.0952224832838032 MinAP: 0.45695976173173014
11%: MAP: 0.9606342778998894 stdAP: 0.0952224832838032 MinAP: 0.45695976173173014
12%: MAP: 0.960826

In [30]:
dt_results = pd.DataFrame(results).T
dt_results.columns = ['MAP', 'stdAP', 'MinAP']
dt_results.to_csv('resultados_recommendation.csv', decimal=',')

,MAP,stdAP,MinAP
0,0.960523,0.095578,0.454644
1,0.960523,0.095578,0.454644
2,0.960523,0.095578,0.454644
3,0.960536,0.095542,0.454644
4,0.960536,0.095542,0.454644
5,0.960536,0.095542,0.454644
6,0.960536,0.095542,0.454644
7,0.960624,0.095250,0.456960
8,0.960634,0.095222,0.456960
9,0.960634,0.095222,0.456960


In [34]:
def average_precision_no_ranking(rank, video_id):
    y_true = []
    y_score = []
    
    for i, r in enumerate(rank):
        y_score.append(1)
        y_true.append(int(r in dt_desc.loc[video_id].rec))
    
    return average_precision_score(y_true, y_score)

In [35]:
results = {}
for i in range(21):
    dt_pct_f = dt_pct.loc[dt_pct.pct >= (i/100)].copy()
    
    average_precision = []

    for video_id in dt_desc.index:
        #pct de cada pessoa
        actor_score = dt_pct_f.loc[dt_pct_f.video_id == video_id][['super_cluster','pct']].set_index('super_cluster')
        #as pessoas
        actors = dt_pct_f.loc[dt_pct_f.video_id == video_id].super_cluster.values
        #pct das pessoas em outro video
        dt_score = dt_pct_f.loc[(dt_pct_f.super_cluster.isin(actors))&(dt_pct_f.video_id != video_id)].copy()
        #score (multiplicacao)
        dt_score['score'] = dt_score.super_cluster.map(actor_score.pct)*dt_score.pct
        #rank com a soma de quem aparece
        rank = dt_score.groupby('video_id')['score'].sum().nlargest(100).index.values
        #average precision
        ap = average_precision_no_ranking(rank, video_id)

        #print(f'id {video_id} = ', ap)


        average_precision.append(ap)
        
    results[i] = [np.mean(average_precision), np.std(average_precision), np.min(average_precision)]
    print(f'{i}%: MAP: {results[i][0]} stdAP: {results[i][1]} MinAP: {results[i][2]}')

0%: MAP: 0.6301927028477103 stdAP: 0.28400015990458727 MinAP: 0.2037037037037037
1%: MAP: 0.6301927028477103 stdAP: 0.28400015990458727 MinAP: 0.2037037037037037
2%: MAP: 0.6321708796024934 stdAP: 0.2823059008461214 MinAP: 0.2037037037037037
3%: MAP: 0.6566384604885256 stdAP: 0.27664326741698003 MinAP: 0.21568627450980393
4%: MAP: 0.6566384604885256 stdAP: 0.27664326741698003 MinAP: 0.21568627450980393
5%: MAP: 0.6812706188692832 stdAP: 0.27145550774534505 MinAP: 0.22448979591836735
6%: MAP: 0.7168614346966868 stdAP: 0.25800722908486073 MinAP: 0.2571428571428571
7%: MAP: 0.7574677147794758 stdAP: 0.23461488213288745 MinAP: 0.3142857142857143
8%: MAP: 0.7630049052915078 stdAP: 0.23409208923682834 MinAP: 0.3142857142857143
9%: MAP: 0.7804352072028251 stdAP: 0.2221960038249193 MinAP: 0.3333333333333333
10%: MAP: 0.8099331652163922 stdAP: 0.212622589073778 MinAP: 0.4090909090909091
11%: MAP: 0.8306414246114967 stdAP: 0.19862616289906754 MinAP: 0.45
12%: MAP: 0.837083861157438 stdAP: 0.1955

In [36]:
dt_results = pd.DataFrame(results).T
dt_results.columns = ['MAP', 'stdAP', 'MinAP']
dt_results.to_csv('resultados_recommendation_no_ranking.csv', decimal=',')

In [38]:
average_precision_score([1,1,1,1,0,1], [1,1,1,1,1,1])

0.8333333333333334